# matchid 100개로 데이터 프레임 만들기

In [1]:
#라이브러리 임포트 하기 

import time
import requests
import pandas as pd


In [305]:
#api_key 
api_key ='RGAPI-c33dbe68-ba64-4996-8a8c-37c7cfd572a0'

# 뽑아둔 매치 아이디 중에서 100개 가져오기 
matchid_fin=pd.read_csv('matchid_final.txt', header=None)
matchid_1=matchid_fin[0:100]
matchid_list=list(matchid_1[0])

# 데이터 프레임 만들기 

### 빈 데이터 프레임 만들기 

In [307]:
## matchid 에 대한 데이터 프레임을 만들어서 그 안에 넣기 
matchid_100=pd.DataFrame()
matchid_100['matchid']=None

#용에 대한 칼럼 만들기 
matchid_100['blue_dragon']=None
matchid_100['blue_firstdragon']=None
matchid_100['blue_HORDE']=None
matchid_100['red_dragon']=None
matchid_100['red_firstdragon']=None
matchid_100['red_HORDE']=None

#퍼블 여부 칼럼 만들기 
matchid_100['blue_firstkill']=None
matchid_100['red_firstkill']=None

#개인별 KDA 칼럼 만들기 
for i in range(1,11):
    matchid_100[f'player{i}kills'] = None
    matchid_100[f'player{i}assists'] = None
    matchid_100[f'player{i}deaths'] = None

matchid_100


,matchid,blue_dragon,blue_firstdragon,blue_HORDE,red_dragon,red_firstdragon,red_HORDE,blue_firstkill,red_firstkill,player1kills,...,player7deaths,player8kills,player8assists,player8deaths,player9kills,player9assists,player9deaths,player10kills,player10assists,player10deaths


### 팀별 퍼스트 블러드 여부, 용 이벤트 , 개인 별 KDA 데이터 프레임에 넣기 

In [308]:
# 각 매치 ID에 대해 반복
for j in range(len(matchid_list)):
    try:
        matchtime_url = f'https://asia.api.riotgames.com/lol/match/v5/matches/{matchid_list[j]}/timeline?api_key={api_key}'
        
        # 매치 타임라인 데이터 요청
        r = requests.get(matchtime_url)
        
        # API 요청이 성공했을 때만 처리
        if r.status_code == 200:
            # 매치 타임라인 데이터를 DataFrame으로 변환
            df_time = pd.DataFrame(r.json()['info']['frames'])
            matchid_100.loc[j,'matchid'] = matchid_list[j]
            blue_elite, red_elite = 0, 0
            blue_HORDE, red_HORDE = 0, 0
            blue_dragon, red_dragon = 0, 0
            blue_firstdragon, red_firstdragon = 0, 0
            blue_firstkill = 0
            red_firstkill = 0
            red_kill = 0
            blue_kill = 0                
            blue_firstkill = 0
            red_firstkill = 0
            red_kill = 0
            blue_kill = 0
            participant_stats={}
                                        

            # 처음 16개 이벤트에 대해서만 반복
            for i in range(0, 16):
                df = pd.DataFrame(df_time['events'][i])
                selected_champion_kills = df[df['type'] == 'CHAMPION_KILL']
                selected_ELITE_MONSTER_KILL = df[df['type'] == 'ELITE_MONSTER_KILL']
                selected_BUILDING_KILL = df[df['type'] == 'BUILDING_KILL']

                
                
                # 킬 이벤트의 통계 계산
                for index, event in selected_champion_kills.iterrows():
                    if event['killerId'] in range(1, 6):  # Blue Team
                        if red_kill == 0 and blue_kill == 0:
                            blue_firstkill += 1
                        blue_kill += 1
                    elif event['killerId'] in range(6, 11):  # Red Team
                        if red_kill == 0 and blue_kill == 0:
                            red_firstkill += 1
                        red_kill += 1

            
                 #용 이벤트 통계 계산        
                for index, event in selected_ELITE_MONSTER_KILL.iterrows():
                    # 이벤트의 유형을 확인합니다.
                    if event['monsterType'] == 'DRAGON':
                        # 킬러의 팀을 확인합니다.
                        if event['killerId'] in range(1, 6):  # Blue Team
                            blue_dragon += 1
                            if blue_dragon == 1 and red_dragon == 0:
                                blue_firstdragon += 1
                            
                        else:  # Red Team
                            red_dragon += 1
                            if red_dragon == 1 and blue_dragon == 0:
                                red_firstdragon += 1
                            
                    elif event['monsterType'] == 'HORDE':
                        if event['killerId'] in range(1, 6):  # Blue Team
                            blue_HORDE += 1
                        else:  # Red Team
                            red_HORDE += 1
                            
                                      

                            
                            
                 # 개인 별 KDA 통계 계산 
                for index, event in selected_champion_kills.iterrows():
                    
                    killer_id =event['killerId']
                    killer_id=int(killer_id)
                    if killer_id not in participant_stats:
                        participant_stats[killer_id] = {'kills': 0, 'assists': 0, 'deaths': 0}
                    
                    participant_stats[killer_id]['kills'] += 1
                    
                    assist_ids = event.get('assistingParticipantIds', [])
                    if isinstance(assist_ids, list):
                        participant_stats[killer_id]['assists'] += len(assist_ids)
                    
                    # 사망자도 킬 수에 포함되므로 사망 수도 증가시킵니다.
                    death_ids = event['victimId']
                    death_ids=int(death_ids)
                    if death_ids not in participant_stats:
                        participant_stats[death_ids] = {'kills': 0, 'assists': 0, 'deaths': 0}
                    
                    participant_stats[death_ids]['deaths'] += 1
                    

            


            # 빈 데이터 프레임에 주어진 변수 넣기              
            matchid_100.loc[j,'red_firstkill'] = red_firstkill
            matchid_100.loc[j,'blue_firstkill'] = blue_firstkill
            matchid_100.loc[j,'red_HORDE'] = red_HORDE
            matchid_100.loc[j,'blue_HORDE'] = blue_HORDE
            matchid_100.loc[j,'blue_dragon'] = blue_dragon
            matchid_100.loc[j,'red_dragon'] = red_dragon
            matchid_100.loc[j,'blue_firstdragon'] = blue_firstdragon
            matchid_100.loc[j,'red_firstdragon'] = red_firstdragon

            
            for k in participant_stats.keys():
                matchid_100.loc[j, f'player{k}kills'] = participant_stats[k]['kills'] 
                matchid_100.loc[j, f'player{k}assists'] = participant_stats[k]['assists']
                matchid_100.loc[j, f'player{k}deaths'] = participant_stats[k]['deaths']
            
            
            

    

    except Exception as e:
        print(f"Error occurred: {e}")
        pass
    
matchid_100

Error occurred: 3


,matchid,blue_dragon,blue_firstdragon,blue_HORDE,red_dragon,red_firstdragon,red_HORDE,blue_firstkill,red_firstkill,player1kills,...,player8deaths,player9kills,player9assists,player9deaths,player10kills,player10assists,player10deaths,player0kills,player0assists,player0deaths
0,KR_6965278983,0,0,3,2,1,3,1,0,1,...,0,4,7,2,1,0,4,NaN,NaN,NaN
1,KR_6961163006,1,1,6,1,0,0,1,0,3,...,3,5,7,4,8,13,1,1.0,0.0,0.0
2,KR_6990258458,0,0,3,1,1,3,0,1,1,...,0,3,4,2,2,2,4,NaN,NaN,NaN
3,KR_6991192961,2,1,2,0,0,4,1,0,NaN,...,5,0,0,2,NaN,NaN,NaN,NaN,NaN,NaN
4,KR_6987828402,2,1,4,0,0,2,0,1,1,...,5,1,2,0,0,0,1,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,KR_6863114183,1,1,0,0,0,0,0,1,4,...,7,3,2,7,3,3,8,1.0,0.0,0.0
96,KR_6980418077,1,1,6,0,0,0,1,0,0,...,0,0,0,3,1,1,2,NaN,NaN,NaN
97,KR_6967213888,0,0,0,1,1,6,1,0,1,...,2,0,0,3,2,3,2,NaN,NaN,NaN
98,KR_6961226279,2,1,2,0,0,4,1,0,0,...,0,2,2,1,3,3,2,NaN,NaN,NaN


In [309]:
# matchid_100 데이터 프레임 확인해보기 
matchid_100.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99 entries, 0 to 99
Data columns (total 42 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   matchid           99 non-null     object 
 1   blue_dragon       98 non-null     object 
 2   blue_firstdragon  98 non-null     object 
 3   blue_HORDE        98 non-null     object 
 4   red_dragon        98 non-null     object 
 5   red_firstdragon   98 non-null     object 
 6   red_HORDE         98 non-null     object 
 7   blue_firstkill    98 non-null     object 
 8   red_firstkill     98 non-null     object 
 9   player1kills      96 non-null     object 
 10  player1assists    96 non-null     object 
 11  player1deaths     96 non-null     object 
 12  player2kills      97 non-null     object 
 13  player2assists    97 non-null     object 
 14  player2deaths     97 non-null     object 
 15  player3kills      98 non-null     object 
 16  player3assists    98 non-null     object 
 17 

## 다른 변수들이 있는 데이터 프레임 합치기 

In [310]:
# Match/V5/matches/info/participants , timeline에 있는 데이터 프레임 가져오기 
df1=pd.read_csv('홍민화.csv')
df1

,matchid,championName_100_1,championName_100_2,championName_100_3,championName_100_4,championName_100_5,championName_200_6,championName_200_7,championName_200_8,championName_200_9,...,jungleMinionsKilled_100_1,jungleMinionsKilled_100_2,jungleMinionsKilled_100_3,jungleMinionsKilled_100_4,jungleMinionsKilled_100_5,jungleMinionsKilled_200_6,jungleMinionsKilled_200_7,jungleMinionsKilled_200_8,jungleMinionsKilled_200_9,jungleMinionsKilled_200_10
0,KR_6965278983,Teemo,Ekko,Tristana,Nilah,Maokai,Ryze,LeeSin,TwistedFate,Ezreal,...,2.0,92.0,0.0,0.0,0.0,4.0,70.0,0.0,0.0,0.0
1,KR_6961163006,Mordekaiser,Vi,Irelia,Smolder,Lux,Volibear,Ekko,Galio,Ezreal,...,0.0,76.0,0.0,0.0,0.0,0.0,84.0,0.0,0.0,0.0
2,KR_6990258458,XinZhao,Volibear,Ahri,Smolder,Braum,Kayle,Nocturne,AurelionSol,Jinx,...,0.0,76.0,0.0,0.0,0.0,0.0,80.0,0.0,0.0,0.0
3,KR_6991192961,KSante,Belveth,Annie,Lucian,Alistar,Riven,Nidalee,Malphite,Senna,...,0.0,102.0,0.0,0.0,0.0,0.0,94.0,0.0,0.0,4.0
4,KR_6987828402,Gangplank,Nidalee,Tristana,Varus,Lulu,Karma,MonkeyKing,Brand,Jinx,...,0.0,88.0,0.0,0.0,0.0,0.0,78.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,KR_6863114183,Aatrox,Belveth,Leblanc,Samira,Senna,Rumble,Graves,Talon,Nilah,...,0.0,70.0,0.0,0.0,0.0,0.0,84.0,0.0,0.0,0.0
94,KR_6980418077,TwistedFate,RekSai,Akshan,Ezreal,Lux,Nasus,Graves,Katarina,Jhin,...,0.0,82.0,1.0,0.0,0.0,0.0,76.0,0.0,0.0,0.0
95,KR_6967213888,Jax,Khazix,Sylas,Draven,Blitzcrank,Yorick,LeeSin,Akali,Ezreal,...,0.0,57.0,0.0,0.0,0.0,0.0,80.0,1.0,0.0,0.0
96,KR_6961226279,KSante,Khazix,Karma,Smolder,Lux,Aatrox,Nidalee,Zed,Varus,...,0.0,76.0,0.0,0.0,0.0,2.0,102.0,0.0,0.0,0.0


In [311]:
#Match/V5/matches 랑 timeline에 있는 데이터 프레임 가져오기 
df2=pd.read_csv('양윤규.csv')
df2

,Unnamed: 0,matchid,team100buildingkill,team200buildingkill,team100wardplaced,team200wardplaced,team100wardkill,team200wardkill,player1totaldamage,player2totaldamage,...,player2magicdamage,player3magicdamage,player4magicdamage,player5magicdamage,player6magicdamage,player7magicdamage,player8magicdamage,player9magicdamage,player10magicdamage,winteam
0,0,KR_6965278983,0,1,33,45,14,7,10506.0,3884.0,...,2855.0,489.0,0.0,3382.0,8017.0,1577.0,4515.0,1453.0,4965.0,[100]
1,1,KR_6961163006,1,0,35,26,8,5,12436.0,3030.0,...,0.0,1950.0,723.0,4491.0,2548.0,4958.0,4583.0,2889.0,6351.0,[100]
2,2,KR_6990258458,0,0,38,29,8,19,7347.0,5234.0,...,1518.0,4227.0,1143.0,3292.0,2894.0,219.0,5690.0,579.0,278.0,[200]
3,3,KR_6991192961,0,0,167,25,3,7,2716.0,6408.0,...,476.0,5997.0,818.0,2044.0,446.0,2439.0,4744.0,0.0,1545.0,[100]
4,4,KR_6987828402,1,1,24,59,7,9,4722.0,4598.0,...,4008.0,1384.0,1270.0,1326.0,5847.0,562.0,6563.0,191.0,1803.0,[100]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,94,KR_6973230618,0,1,57,298,11,6,8880.0,2345.0,...,1987.0,4808.0,336.0,101.0,903.0,512.0,4418.0,412.0,0.0,[200]
91,95,KR_6863114183,3,0,30,28,6,12,9234.0,4622.0,...,668.0,8295.0,1508.0,0.0,4040.0,176.0,0.0,0.0,3200.0,[100]
92,96,KR_6980418077,8,0,30,36,4,5,6190.0,4385.0,...,776.0,655.0,1505.0,7508.0,8171.0,224.0,2689.0,130.0,2189.0,[100]
93,98,KR_6961226279,0,3,58,28,12,9,6348.0,3366.0,...,102.0,6248.0,179.0,5383.0,1853.0,8239.0,471.0,1180.0,4264.0,NaN


In [312]:
# 데이터 프레임 두 개 합치기 
merged_df = pd.merge(df1, df2, on='matchid', how='outer')

In [313]:
# 데이터 프레임 두 개 합치기 
merged_df2=pd.merge(merged_df,matchid_100, on='matchid', how='outer')

# 최종 데이터 프레임 확인

In [314]:
# 최종 데이터 프레임 확인하기 
merged_df2

,matchid,championName_100_1,championName_100_2,championName_100_3,championName_100_4,championName_100_5,championName_200_6,championName_200_7,championName_200_8,championName_200_9,...,player8deaths,player9kills,player9assists,player9deaths,player10kills,player10assists,player10deaths,player0kills,player0assists,player0deaths
0,KR_6965278983,Teemo,Ekko,Tristana,Nilah,Maokai,Ryze,LeeSin,TwistedFate,Ezreal,...,0,4,7,2,1,0,4,NaN,NaN,NaN
1,KR_6961163006,Mordekaiser,Vi,Irelia,Smolder,Lux,Volibear,Ekko,Galio,Ezreal,...,3,5,7,4,8,13,1,1.0,0.0,0.0
2,KR_6990258458,XinZhao,Volibear,Ahri,Smolder,Braum,Kayle,Nocturne,AurelionSol,Jinx,...,0,3,4,2,2,2,4,NaN,NaN,NaN
3,KR_6991192961,KSante,Belveth,Annie,Lucian,Alistar,Riven,Nidalee,Malphite,Senna,...,5,0,0,2,NaN,NaN,NaN,NaN,NaN,NaN
4,KR_6987828402,Gangplank,Nidalee,Tristana,Varus,Lulu,Karma,MonkeyKing,Brand,Jinx,...,5,1,2,0,0,0,1,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,KR_6980418077,TwistedFate,RekSai,Akshan,Ezreal,Lux,Nasus,Graves,Katarina,Jhin,...,0,0,0,3,1,1,2,NaN,NaN,NaN
95,KR_6967213888,Jax,Khazix,Sylas,Draven,Blitzcrank,Yorick,LeeSin,Akali,Ezreal,...,2,0,0,3,2,3,2,NaN,NaN,NaN
96,KR_6961226279,KSante,Khazix,Karma,Smolder,Lux,Aatrox,Nidalee,Zed,Varus,...,0,2,2,1,3,3,2,NaN,NaN,NaN
97,KR_6654208256,Renekton,Hecarim,Akali,Ezreal,Xerath,Fiora,Nocturne,Zilean,Zeri,...,5,2,2,4,3,0,5,NaN,NaN,NaN


In [315]:
merged_df2.to_csv('최최종100.csv', index=False)  